In [25]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

iris = load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["target"] = iris.target
df = df[df["target"].isin([1, 2])].reset_index(drop=True)

In [26]:
class LogRegCust:
    def __init__(self, learning_rate=0.01, n_iter=1000, tol=0.001, verbose=False):
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.tol = tol
        self.verbose = verbose
        self.weights = None
        self.bias = None
        self.classes = None
        self.losses = []

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def one_hot_encode(self, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_samples = len(y)

        encoded = np.zeros((n_samples, n_classes))
        for i, cls in enumerate(self.classes):
            encoded[:, i] = (y == cls).astype(int)
        return encoded

    def binary_cross_entropy(self, y_true, y_pred):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def fit(self, X, y):
        X = X.values if hasattr(X, "values") else np.array(X)
        y = y.values if hasattr(y, "values") else np.array(y)

        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        y_binary = np.where(y == self.classes[0], 0, 1)

        self.weights = np.zeros(n_features)
        self.bias = 0

        # градиентный спуск
        for iteration in range(self.n_iter):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear_model)

            loss = self.binary_cross_entropy(y_binary, y_pred)
            self.losses.append(loss)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y_binary))
            db = (1 / n_samples) * np.sum(y_pred - y_binary)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict_prob(self, X):
        X = X.values if hasattr(X, "values") else np.array(X)
        linear_model = np.dot(X, self.weights) + self.bias
        prob = self.sigmoid(linear_model)
        return prob

    def predict(self, X, threshold=0.5):
        prob = self.predict_prob(X)
        pred = np.where(prob >= threshold, self.classes[1], self.classes[0])
        return pred

    def score(self, X, y):
        pred = self.predict(X)
        y = y.values if hasattr(y, "values") else np.array(y)
        return np.mean(pred == y)

In [27]:
X = df[iris.feature_names]
y = df["target"]

model = LogRegCust(learning_rate=0.1, n_iter=1000, verbose=True)
model.fit(X, y)

print(model.score(X, y))

0.97
